# Alternativas gratuitas para ChatBots

Hay **cientos** de modelos de lenguaje grande (large language models LLM) disponibles.
En este enlace se encuentra organizadon según su desempeño y funciones:

https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard


Hay que tener en cuenta que estos modelos, principalmente los que tienen mejor desempeño, suelen ser muy pesados, y su tiempo de inferencia (el tiempo para generar texto) puede ser muy alto si no se ejecuta en GPU. Este alto tiempo es el **precio a pagar por usar un servicio gratuito**.

De estos modelos en este notebook se usará el modelo [Qwen2](https://huggingface.co/Qwen/Qwen2-1.5B-Instruct). Este modelo pesa aproximadamente 3GB, da respuestas razonablemente buenas, y su tiempo de inferencia puede ser de hasta 5 minutos si se ejecuta en CPU.

En este notebook se ejecutará en ChatBot en una página web en local, para ellos es necesario:
  
1.   Registrarse en [ngrok](https://dashboard.ngrok.com/signup)
2.   Crear un token en el [dashboard](https://dashboard.ngrok.com/get-started/your-authtoken)


In [1]:
!pip install pyngrok
from pyngrok import ngrok

# Configurar tu token de autenticación
NGROK_AUTH_TOKEN = "2inSmrIDemwkTsSyTluD5fTUbjQ_4AsmxyZBAXH1PQB6mRD9m"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer

# Inicializar la aplicación Flask
app = Flask(__name__)

# Cargar el modelo y el tokenizador en la CPU por defecto
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

# Historial de preguntas y respuestas
historial = []

# Función para generar la respuesta
def generar_respuesta(prompt):
    # Configurar los mensajes con el prompt del usuario
    messages = [
        {"role": "Chatbot", "content": ""},
        {"role": "Usuario", "content": prompt + ' \n'}
    ]

    # Aplicar el template de chat al prompt
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenizar y preparar los inputs para el modelo
    model_inputs = tokenizer([text], return_tensors="pt")

    # Generar texto con el modelo
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_length=512  # Cambiado de max_new_tokens a max_length para definir la longitud máxima de la generación
    )

    # Decodificar la salida generada
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return generated_text

# Ruta principal para renderizar el formulario HTML
@app.route('/')
def index():
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Generador de Respuestas AI</title>
    </head>
    <body>
        <h1>Generador de Respuestas AI</h1>
        <form action="/ask" method="post">
            <label for="question">Ingresa tu pregunta:</label><br>
            <input type="text" id="question" name="question" required><br><br>
            <input type="submit" value="Enviar">
        </form>

        {% if historial %}
            {% for entry in historial %}
                <h2>Pregunta:</h2>
                <p>{{ entry.pregunta }}</p>
                <h2>Respuesta:</h2>
                <p>{{ entry.respuesta }}</p>
            {% endfor %}
        {% endif %}
    </body>
    </html>
    ''', historial=historial)

# Ruta para manejar las respuestas del formulario
@app.route('/ask', methods=['POST'])
def ask():
    if request.method == 'POST':
        prompt = request.form['question']
        respuesta_generada = generar_respuesta(prompt)
        historial.append({"pregunta": prompt, "respuesta": respuesta_generada})
        return render_template_string('''
        <!DOCTYPE html>
        <html>
        <head>
            <title>Generador de Respuestas AI</title>
        </head>
        <body>
            <h1>Generador de Respuestas AI</h1>
            <form action="/ask" method="post">
                <label for="question">Ingresa tu pregunta:</label><br>
                <input type="text" id="question" name="question" required><br><br>
                <input type="submit" value="Enviar">
            </form>

            {% if historial %}
                {% for entry in historial %}
                    <h2>Pregunta:</h2>
                    <p>{{ entry.pregunta }}</p>
                    <h2>Respuesta:</h2>
                    <p>{{ entry.respuesta }}</p>
                {% endfor %}
            {% endif %}
        </body>
        </html>
        ''', historial=historial)

# Iniciar ngrok y ejecutar la aplicación Flask
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    app.run()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 * ngrok tunnel "NgrokTunnel: "https://026a-34-127-116-112.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 21:05:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 21:05:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 21:09:44] "POST /ask HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2024 21:15:31] "POST /ask HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jul/2024 00:03:53] "POST /ask HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jul/2024 00:06:42] "POST /ask HTTP/1.1" 200 -


# Actividad

Elegir un modelo pre-entrenado y crear su chatbot, puede ejecutarse en Colab (o en consola si se ejecuta en local), o en un sitio web local como en este notebook.